In [2]:
import pandas as pd
import numpy as np
import os
os.environ['MKL_NUM_THREADS'] = '1'
from collections import Counter
import itertools
import json
import scipy.sparse as sparse
import pickle
from torch.utils.data import DataLoader
from implicit.als import AlternatingLeastSquares as WMF
from implicit.bpr import BayesianPersonalizedRanking as BPR
from implicit import _als
from eval.rec_eval import *
from torch.utils.tensorboard import SummaryWriter
import datetime
import importlib

import torch

In [3]:
dataset_name='ml-1m-l-1-100'
with open(os.path.join("data", "parsed", dataset_name), 'rb') as f:
    tr, val, te = pickle.load(f)

In [4]:
from SLIM import SLIM, SLIMatrix
n_users, n_items =tr.shape

In [5]:
import pandas as pd
from SLIM import SLIM, SLIMatrix

#read training data stored as triplets <user> <item> <rating>
trainmat = SLIMatrix(tr)

In [6]:
trainmat

In [7]:
params = {'algo':'cd', 'simtype': 'dotp', 'nthreads':8, 'l1r':1.0, 'l2r':1.0}

#learn the model using training data and desired parameters
model = SLIM()
model.train(params, trainmat)

Learning takes 23.146 secs.


In [8]:
model.save_model(modelfname='model.csr', # filename of the model
                 mapfname='map.csr')

In [9]:
cat map.csr

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225

227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
2

In [10]:
model_new = SLIM()
model_new.load_model(modelfname='model.csr', # filename of the model
                 mapfname='map.csr' # filename of the item map
                )


In [11]:
from scipy.sparse import *
spmat = dok_matrix((n_items, n_items))

In [12]:
row = []
col = []

In [13]:
class wrapper():
    def __init__(self, spmat):
        self.csr = spmat.tocsc()
    
    def rank_items(self, user, tr, val):
        scores = (tr[user] * self.csr[:, val]).todense().A1
        #scores = ((self.csr[val] * tr[user].T).T).todense().A1
        i = np.argsort(-scores)
        s = scores[i].tolist()
        i = val[i].tolist()
        return list(zip(i, i))

In [14]:
with open("model.csr", 'r') as f:
    l = "a"
    dd = 0
    while l != "":
        l = f.readline()
        line = l.strip().split()
        i = [int(x) for (k, x) in enumerate(line) if k % 2 == 0]
        v = [float(x) for (k, x) in enumerate(line) if k % 2 == 1]
        for key, vv in zip(i,v):
            spmat[dd, key] = vv
        dd += 1

In [15]:
wr = wrapper(spmat)

In [16]:
wr.rank_items(3, tr, np.asarray([1,2,3, 4, 5, 6]))

[(4, 4), (6, 6), (5, 5), (2, 2), (3, 3), (1, 1)]

In [17]:
dataset_name='ml-1m-l-1-100'
with open(os.path.join("data", "parsed", dataset_name), 'rb') as f:
    tr, val, te = pickle.load(f)

In [19]:
import time
b = time.time()
leave_k_eval(wr, tr, val, leavek=1, K=10, num_threads=4)
print(time.time() - b)

2.688096046447754


In [18]:
import time
b = time.time()
leave_k_eval(wr, tr, val, leavek=1, K=10)

{'precision': 0.7951607557176003,
 'map': 0.49448502138674705,
 'ndcg': 0.5664438238995964}